In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import cv2
from tqdm import tqdm
import wandb
from datetime import datetime
import os
import sys

# 프로젝트 경로 추가
project_path = Path('C:/EngineBladeAI/EngineInspectionAI_MS')
sys.path.append(str(project_path))

# Custom imports
from models.heads.mask2former_damage_head import Mask2FormerDamageHead
from models.backbones.convnext_fpn import ConvNeXtFPN
from utils.evaluate import ModelEvaluator

# Device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    free_memory = (torch.cuda.get_device_properties(0).total_memory - 
                   torch.cuda.memory_allocated(0)) / 1024**3
    print(f"   GPU: {gpu_name}")
    print(f"   Total Memory: {gpu_memory:.1f} GB")
    print(f"   Free Memory: {free_memory:.1f} GB")

print("\n✅ Environment setup complete!")

SyntaxError: invalid syntax (convnext_fpn.py, line 113)

In [ ]:
class Config:
    """프로젝트 전체 설정"""
    
    # 손상 클래스 정의
    NUM_CLASSES = 3
    CLASS_NAMES = ['crack', 'nick', 'tear']
    CLASS_COLORS = {
        'crack': (255, 0, 0),    # Red
        'nick': (0, 255, 0),      # Green
        'tear': (0, 0, 255),      # Blue
    }
    
    # 데이터 경로
    DATA_ROOT = project_path / 'data'
    TRAIN_DATA = DATA_ROOT / 'multilabeled_data_augmented' / 'train'
    VALID_DATA = DATA_ROOT / 'multilabeled_data_augmented' / 'valid'
    TEST_DATA = DATA_ROOT / 'multilabeled_data_augmented' / 'test'
    
    # 모델 설정 (RTX 4090 최적화)
    QUERIES_PER_CLASS = 100  # 각 클래스당 100개
    TOTAL_QUERIES = NUM_CLASSES * QUERIES_PER_CLASS  # 300
    
    # 백본 설정
    BACKBONE_MODEL = 'convnext_tiny'
    BACKBONE_PRETRAINED = True
    FPN_CHANNELS = 256
    
    # Mask2Former 설정
    HIDDEN_DIM = 256
    NUM_HEADS = 8
    DIM_FEEDFORWARD = 1024
    DEC_LAYERS = 3
    DROPOUT = 0.1
    
    # 학습 설정
    BATCH_SIZE = 2
    GRAD_ACCUM_STEPS = 2  # Effective batch size = 4
    NUM_EPOCHS = 50
    LEARNING_RATE = 5e-5
    WEIGHT_DECAY = 0.01
    
    # Loss 가중치
    LOSS_CE_WEIGHT = 2.0
    LOSS_MASK_WEIGHT = 5.0
    LOSS_DICE_WEIGHT = 2.0
    
    # 기타
    NUM_WORKERS = 4
    CHECKPOINT_DIR = project_path / 'checkpoints' / f'run_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    USE_WANDB = False
    WANDB_PROJECT = 'EngineBladeAI'
    
config = Config()
print("Configuration loaded:")
print(f"  Classes: {config.CLASS_NAMES}")
print(f"  Total Queries: {config.TOTAL_QUERIES}")
print(f"  Batch Size: {config.BATCH_SIZE} (×{config.GRAD_ACCUM_STEPS} = {config.BATCH_SIZE * config.GRAD_ACCUM_STEPS})")
print(f"  Epochs: {config.NUM_EPOCHS}")

In [ ]:
def create_mask2former_model(config):
    """Mask2Former 모델 생성 (300 queries, 3 classes)"""
    
    model = Mask2FormerDamageHead(
        in_channels=config.FPN_CHANNELS,
        num_classes=config.NUM_CLASSES,  # 3
        num_queries=config.TOTAL_QUERIES,  # 300
        hidden_dim=config.HIDDEN_DIM,
        num_heads=config.NUM_HEADS,
        dim_feedforward=config.DIM_FEEDFORWARD,
        dec_layers=config.DEC_LAYERS,
        dropout=config.DROPOUT,
        use_blade_mask=True
    )
    
    # Background 클래스 제거 (num_classes + 1 → num_classes)
    if hasattr(model, 'class_head'):
        if model.class_head.out_features != config.NUM_CLASSES:
            print(f"⚠️ Fixing class_head: {model.class_head.out_features} → {config.NUM_CLASSES}")
            in_features = model.class_head.in_features
            model.class_head = nn.Linear(in_features, config.NUM_CLASSES)
    
    # Query-to-class 매핑 추가
    model.query_to_class = {}
    for q in range(config.TOTAL_QUERIES):
        model.query_to_class[q] = q // config.QUERIES_PER_CLASS
    
    # Class specialization 함수
    def apply_class_specialization(outputs):
        """각 query가 자신의 전문 클래스만 높게 예측"""
        pred_logits = outputs['pred_logits']
        batch_size, num_queries, num_classes = pred_logits.shape
        
        for q in range(num_queries):
            assigned_class = model.query_to_class.get(q, 0)
            if assigned_class < num_classes:
                # 해당 클래스는 증폭, 다른 클래스는 억제
                mask = torch.ones_like(pred_logits[:, q])
                mask[:, assigned_class] *= 5.0
                for c in range(num_classes):
                    if c != assigned_class:
                        mask[:, c] *= 0.1
                outputs['pred_logits'][:, q] = outputs['pred_logits'][:, q] * mask
        
        return outputs
    
    model.apply_class_specialization = apply_class_specialization
    
    return model

# 모델 생성
print("\n" + "="*50)
print("Creating Mask2Former model...")
print("="*50)

mask2former = create_mask2former_model(config).to(device)

# 파라미터 수 확인
num_params = sum(p.numel() for p in mask2former.parameters()) / 1e6
trainable_params = sum(p.numel() for p in mask2former.parameters() if p.requires_grad) / 1e6

print(f"✅ Model created!")
print(f"   Total parameters: {num_params:.2f}M")
print(f"   Trainable parameters: {trainable_params:.2f}M")
print(f"   Queries: {config.TOTAL_QUERIES} (Crack:0-99, Nick:100-199, Tear:200-299)")
print(f"   Output classes: {config.NUM_CLASSES}")

In [ ]:
print("\n" + "="*50)
print("Creating ConvNeXt-FPN backbone...")
print("="*50)

backbone = ConvNeXtFPN(
    model_name=config.BACKBONE_MODEL,
    pretrained=config.BACKBONE_PRETRAINED,
    feature_channels=[96, 192, 384, 768],  # ConvNeXt-Tiny
    fpn_channels=config.FPN_CHANNELS
).to(device)

print(f"✅ Backbone created: {config.BACKBONE_MODEL}")
print(f"   FPN channels: {config.FPN_CHANNELS}")